# Formulación matemática del Caso 3: CVRP con Estaciones de Recarga, Peajes y Restricciones de Peso

## 1. Formulación del Modelo

Este modelo extiende el problema clásico de ruteo de vehículos (CVRP) incluyendo decisiones de recarga de combustible en estaciones con distintos precios, restricciones de peso por municipio y costos de peaje que varían según el peso transportado. Se busca minimizar el costo asociado al recorrido (distancia), al combustible recargado y a los peajes, asegurando cobertura de demanda y restricciones operacionales y normativas.

### 1.1. Conjuntos

- $L = \{1, \dots, n\}$: Conjunto de localidades (puerto, destinos, estaciones).
- $D = \{2, \dots, d\}$: Conjunto de destinos (clientes).
- $V = \{1, \dots, m\}$: Conjunto de vehículos.
- $E = \{d+2, \dots, n\}$: Conjunto de estaciones de recarga.
- $P = \{1\}$: Puerto (depósito inicial y final).

### 1.2. Índices

- $i, j \in L$: Localidades.
- $k \in V$: Vehículos.

### 1.3. Parámetros

- $distancias_{ij}$: Distancia entre $i$ y $j$.
- $D\_demanda_i$: Demanda del cliente $i$.
- $D\_peso\_max_i$: Peso máximo permitido en el municipio $i$.
- $V\_capacidad_k$: Capacidad del vehículo $k$.
- $V\_autonomia_k$: Autonomía del vehículo $k$.
- $E\_costo_i$: Costo del combustible en estación $i$.
- $T\_base_{ij}$: Tarifa base del peaje entre $i$ y $j$.
- $T\_peso_{ij}$: Tarifa por tonelada en el tramo $i \rightarrow j$.
- $costo$: Costo por kilómetro (tarifa flete + mantenimiento).

### 1.4. Variables de decisión

- $x_{ijk} \in \{0,1\}$: 1 si el vehículo $k$ viaja de $i$ a $j$.
- $u_{ik} \in \mathbb{Z}$: Orden de visita para eliminación de subtours.
- $c_{ik} \geq 0$: Combustible al llegar a $i$.
- $r_{ik} \geq 0$: Combustible recargado en estación $i$.
- $w_{ik} \geq 0$: Peso total que transporta el vehículo $k$ al llegar a $i$.

---

## 2. Función Objetivo

Minimizar el costo total de transporte, combustible y peajes:

$$
\min \sum_{k \in V} \sum_{i \in L} \sum_{\substack{j \in L \\ j \ne i}} \left( costo \cdot distancias_{ij} \cdot x_{ijk} \right) + \sum_{k \in V} \sum_{i \in E} E\_costo_i \cdot r_{ik} + \sum_{k \in V} \sum_{i \in L} \sum_{\substack{j \in L \\ j \ne i}} x_{ijk} \cdot \left(T\_base_{ij} + T\_peso_{ij} \cdot w_{ik} \right)
$$

---

## 3. Restricciones

### (1) Cada cliente es visitado una única vez:

$$
\sum_{k \in V} \sum_{\substack{i \in L \\ i \ne j}} x_{ijk} = 1 \quad \forall j \in D
$$

### (2) Salida desde el puerto:

$$
\sum_{j \in D} x_{1jk} = 1 \quad \forall k \in V
$$

### (3) Retorno al puerto:

$$
\sum_{\substack{i \in L \\ i \ne 1}} x_{i1k} = 1 \quad \forall k \in V
$$

### (4) Conservación de flujo:

$$
\sum_{\substack{i \in L \\ i \ne h}} x_{ihk} = \sum_{\substack{j \in L \\ j \ne h}} x_{hjk} \quad \forall h \in L \setminus \{1\}, \forall k \in V
$$

### (5) Eliminación de subtours (MTZ):

$$
u_{ik} - u_{jk} + n \cdot x_{ijk} \leq n - 1 \quad \forall i \ne j \in L \setminus \{1\}, \forall k \in V
$$

### (6) Capacidad del vehículo:

$$
\sum_{i \in D} D\_demanda_i \cdot \sum_{\substack{j \in L \\ j \ne i}} x_{jik} \leq V\_capacidad_k \quad \forall k \in V
$$

### (7) Dinámica del combustible:

$$
c_{jk} \geq c_{ik} + r_{ik} - distancias_{ij} \cdot x_{ijk} \quad \forall i \ne j \in L, \forall k \in V
$$

### (8) Capacidad del tanque:

$$
c_{ik} \leq V\_autonomia_k \quad \forall i \in L, \forall k \in V
$$

$$
r_{ik} \leq V\_autonomia_k \quad \forall i \in E, \forall k \in V
$$

### (9) Restricción de peso por municipio:

$$
w_{ik} \leq D\_peso\_max_i \quad \forall i \in D \text{ con } D\_peso\_max_i > 0, \forall k \in V
$$

### (10) Cálculo del peso al llegar al municipio:

$$
w_{ik} = \sum_{\substack{j \in L \\ j \ne i}} D\_demanda_i \cdot x_{jik} \quad \forall i \in D, \forall k \in V
$$

---

Este modelo garantiza cobertura de clientes, capacidad y autonomía de vehículos, decisiones óptimas de recarga, cumplimiento de las restricciones de peso por municipio y cálculo de peajes de acuerdo con el peso transportado.


In [1]:
from geopy.distance import geodesic
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

# ----------------------------
# Lectura de datos
# ----------------------------

clientes = pd.read_csv('Proyecto_C_Caso3/clients.csv')
depositos = pd.read_csv('Proyecto_C_Caso3/depots.csv')
estaciones = pd.read_csv('Proyecto_C_Caso3/stations.csv')
vehiculos = pd.read_csv('Proyecto_C_Caso3/vehicles.csv')
tolls = pd.read_csv('Proyecto_C_Caso3/tolls.csv')

# Normalizar nombres de columnas para evitar errores
cols = [col.strip().lower() for col in tolls.columns]
tolls.columns = cols

# Preparación de localidades
locations_df = pd.read_csv('Proyecto_C_Caso3/locations_initial.csv')
locations_df = locations_df[locations_df['LocationID'] < 16]

# Sobrescribimos primero con las localidades iniciales
locations_df.to_csv('Proyecto_C_Caso3/locations.csv', index=False)

# Luego agregamos estaciones (sin encabezado)
for i in range(len(estaciones)):
    data = {
        'LocationID': estaciones['LocationID'][i],
        'Longitude': estaciones['Longitude'][i],
        'Latitude': estaciones['Latitude'][i],
    }
    pd.DataFrame([data]).to_csv('Proyecto_C_Caso3/locations.csv', mode='a', header=False, index=False)

locations_csv = pd.read_csv('Proyecto_C_Caso3/locations.csv')

# Convertir a numérico por seguridad
locations_csv['Latitude'] = pd.to_numeric(locations_csv['Latitude'], errors='coerce')
locations_csv['Longitude'] = pd.to_numeric(locations_csv['Longitude'], errors='coerce')

# Calcular distancias entre localidades
distancias = []
for i in range(len(locations_csv)):
    coord1 = (locations_csv['Latitude'][i], locations_csv['Longitude'][i])
    fila = []
    for j in range(len(locations_csv)):
        coord2 = (locations_csv['Latitude'][j], locations_csv['Longitude'][j])
        fila.append(geodesic(coord1, coord2).kilometers)
    distancias.append(fila)

# Procesar costos de peajes
costo_peaje = {}
for _, row in tolls.iterrows():
    origen = int(row['clientid'])
    destino = int(row['clientid'])
    base = 0 if pd.isna(row['baserate']) else float(str(row['baserate']).replace(",", ""))
    por_tonelada = 0 if pd.isna(row['rateperton']) else float(str(row['rateperton']).replace(",", ""))
    costo_peaje[(origen, destino)] = (base, por_tonelada)



In [2]:
# Parámetros y conjuntos
num_puertos = len(depositos)
num_clientes = len(clientes)
num_localidades = len(locations_csv)
num_vehiculos = len(vehiculos)
num_estaciones = len(estaciones)

Model = ConcreteModel()

P = RangeSet(1, num_puertos)
D = RangeSet(2, num_clientes + 1)
V = RangeSet(1, num_vehiculos)
E = RangeSet(num_clientes + 2, num_localidades)
L = RangeSet(1, num_localidades)
N = RangeSet(2, num_localidades)

D_demanda = {i + 2: clientes['Demand'][i] for i in range(num_clientes)}
D_peso_max = {i + 2: clientes['MaxWeight'][i] if not pd.isna(clientes['MaxWeight'][i]) else float('inf') for i in range(num_clientes)}
V_capacidad = {i + 1: vehiculos['Capacity'][i] for i in range(num_vehiculos)}
V_autonomia = {i + 1: vehiculos['Range'][i] for i in range(num_vehiculos)}
E_costo = {i + num_clientes + 2: estaciones['FuelCost'][i] for i in range(num_estaciones)}

# Variables
Model.x = Var(L, L, V, domain=Binary)
Model.u = Var(N, V, bounds=(1, num_localidades - 1), domain=Integers)
Model.c = Var(L, V, domain=NonNegativeReals)
Model.r = Var(E, V, domain=NonNegativeReals)
Model.peaje = Var(L, L, V, domain=NonNegativeReals)

# Costos
flete = 5000
mant = 700
costo_km = flete + mant

# Objetivo
Model.obj = Objective(
    expr=sum(costo_km * distancias[i-1][j-1] * Model.x[i, j, k] for i in L for j in L for k in V if i != j) +
         sum(E_costo[e] * Model.r[e, k] for e in E for k in V) +
         sum(Model.peaje[i, j, k] for i in L for j in L for k in V if i != j),
    sense=minimize
)

# Restricciones

# Restricción para calcular costos de peaje con activación lineal
Model.peso_total = Var(L, V, domain=NonNegativeReals)
Model.res_peajes = ConstraintList()
M = 1e6  # constante grande
for i in L:
    for j in L:
        if i != j:
            for k in V:
                base, por_ton = costo_peaje.get((i, j), (0, 0))
                Model.res_peajes.add(
                    Model.peaje[i, j, k] <= base * Model.x[i, j, k] + por_ton * Model.peso_total[j, k]
                )
                Model.res_peajes.add(Model.peaje[i, j, k] >= 0)
Model.res1 = ConstraintList()
for j in D:
    Model.res1.add(sum(Model.x[i, j, k] for i in L if i != j for k in V) == 1)

Model.res2 = ConstraintList()
for k in V:
    Model.res2.add(sum(Model.x[1, j, k] for j in D) == 1)

Model.res3 = ConstraintList()
for k in V:
    Model.res3.add(sum(Model.x[i, 1, k] for i in L if i != 1) == 1)

Model.res4 = ConstraintList()
for k in V:
    for h in L:
        if h != 1:
            Model.res4.add(sum(Model.x[i, h, k] for i in L if i != h) == sum(Model.x[h, j, k] for j in L if j != h))

Model.res5 = ConstraintList()
for k in V:
    for i in N:
        for j in N:
            if i != j:
                Model.res5.add(Model.u[i, k] - Model.u[j, k] + num_localidades * Model.x[i, j, k] <= num_localidades - 1)

Model.res6 = ConstraintList()
for k in V:
    Model.res6.add(sum(D_demanda[i] * sum(Model.x[j, i, k] for j in L if j != i) for i in D) <= V_capacidad[k])

Model.res7 = ConstraintList()
for k in V:
    for i in L:
        for j in L:
            if i != j:
                recarga = Model.r[i, k] if i in E else 0
                Model.res7.add(Model.c[j, k] >= Model.c[i, k] + recarga - distancias[i-1][j-1] * Model.x[i, j, k])

Model.res8 = ConstraintList()
for k in V:
    for i in L:
        Model.res8.add(Model.c[i, k] <= V_autonomia[k])
    for i in E:
        Model.res8.add(Model.r[i, k] <= V_autonomia[k])

Model.res9 = ConstraintList()
for j in D:
    peso_max = D_peso_max[j]
    for k in V:
        Model.res9.add(sum(D_demanda[i] * Model.x[i, j, k] for i in D if i != j) <= peso_max)

# Restricción: calcular el peso total al llegar a cada cliente
Model.res_peso_total = ConstraintList()
for j in D:
    for k in V:
        Model.res_peso_total.add(
            Model.peso_total[j, k] == sum(D_demanda[i] * Model.x[i, j, k] for i in D if i != j)
        )

# Resolución
solver = SolverFactory('glpk')
solver.options['tmlim'] = 300
results = solver.solve(Model, tee=True)

# Verificaciones clave
print("Claves de D_demanda:", list(D_demanda.keys()))
print("Claves de D_peso_max:", list(D_peso_max.keys()))

for k in V:
    print(f"Rutas vehículo {k}:")
    for i in L:
        for j in L:
            if i != j and Model.x[i, j, k].value and Model.x[i, j, k].value > 0.5:
                print(f"  Va de {i} a {j}")

print("Modelo listo para resolver el Caso 3.")




GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 300 --write C:\Users\Mariana\AppData\Local\Temp\tmp53pt77mq.glpk.raw
 --wglp C:\Users\Mariana\AppData\Local\Temp\tmpqfrm8w4j.glpk.glp --cpxlp C:\Users\Mariana\AppData\Local\Temp\tmpj3uoc9s7.pyomo.lp
Reading problem data from 'C:\Users\Mariana\AppData\Local\Temp\tmpj3uoc9s7.pyomo.lp'...
C:\Users\Mariana\AppData\Local\Temp\tmpj3uoc9s7.pyomo.lp:118313: warning: lower bound of variable 'x2' redefined
C:\Users\Mariana\AppData\Local\Temp\tmpj3uoc9s7.pyomo.lp:118313: warning: upper bound of variable 'x2' redefined
17096 rows, 8898 columns, 49464 non-zeros
4368 integer variables, 4212 of which are binary
122525 lines were read
Writing problem data to 'C:\Users\Mariana\AppData\Local\Temp\tmpqfrm8w4j.glpk.glp'...
105499 lines were written
GLPK Integer Optimizer 5.0
17096 rows, 8898 columns, 49464 non-zeros
4368 integer variables, 4212 of which are binary
Preprocessing...
12 hidden packing inequaliti(es) were dete

In [6]:
def exportar_resultados_vehiculos_case3(Model, distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje, velocidad=50):
    columnas = [
        'VehicleId', 'LoadCap', 'FuelCap', 'RouteSeq', 'Municipalities', 'Demand',
        'InitLoad', 'InitFuel', 'RefuelStops', 'RefuelAmounts', 'TollsVisited', 'TollCosts',
        'VehicleWeights', 'Distance', 'Time', 'FuelCost', 'TollCost', 'TotalCost'
    ]
    resultados = []

    for k in V:
        ruta = [1]
        actual = 1
        while True:
            siguiente = None
            for j in L:
                if j != actual and Model.x[actual, j, k].value and Model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    ruta.append(j)
                    actual = j
                    break
            if siguiente is None or actual == 1:
                break

        ruta_nombres = ['PTO'] + [
            f"MUN{str(nodo).zfill(2)}" if nodo in D else
            f"EST{str(nodo).zfill(2)}" if nodo in E else
            f"PEA{str(nodo).zfill(2)}" for nodo in ruta[1:-1]
        ] + ['PTO']

        municipios = [n for n in ruta if n in D_demanda]
        demandas = [D_demanda[n] for n in municipios]
        total_demanda = sum(demandas)
        distancia_total = sum(distancias[ruta[i]-1][ruta[i+1]-1] for i in range(len(ruta)-1))
        tiempo = round(distancia_total / velocidad, 2)

        refuel_stops = [i for i in ruta if i in E and Model.r[i, k].value > 0.1]
        refuel_amounts = [round(Model.r[i, k].value, 2) for i in refuel_stops]
        fuel_cost = round(sum(Model.r[i, k].value * E_costo[i] for i in E if Model.r[i, k].value > 0.1), 2)

        tolls_visited = [(ruta[i], ruta[i+1]) for i in range(len(ruta)-1) if (ruta[i], ruta[i+1]) in costo_peaje]
        toll_costs = []
        for (i, j) in tolls_visited:
            base, por_ton = costo_peaje.get((i, j), (0, 0))
            if (j in D) and ((j, k) in Model.peso_total):
                peso_real = Model.peso_total[j, k].value
            else:
                peso_real = 0
            costo = base + por_ton * peso_real
            toll_costs.append(round(costo, 2))
        toll_total = round(sum(toll_costs), 2)

        pesos_por_municipio = [int(D_demanda[m]) for m in municipios]

        total_cost = round(distancia_total * (flete + mant) + fuel_cost + toll_total)

        resultados.append([
            f"CAM{str(k).zfill(3)}",
            V_capacidad[k],
            V_autonomia[k],
            ' - '.join(ruta_nombres),
            len(municipios),
            '-'.join(str(int(d)) for d in demandas),
            total_demanda,
            V_autonomia[k],
            len(refuel_stops),
            '-'.join(str(a) for a in refuel_amounts) if refuel_amounts else '0',
            len(tolls_visited),
            '-'.join(str(tc) for tc in toll_costs) if toll_costs else '0',
            '-'.join(str(p) for p in pesos_por_municipio),
            round(distancia_total, 1),
            tiempo,
            fuel_cost,
            toll_total,
            total_cost
        ])

    df_resultados = pd.DataFrame(resultados, columns=columnas)
    df_resultados.to_csv("Proyecto_C_Caso3/verificacion_caso3.csv", index=False)
    print("Archivo verificacion_caso3.csv exportado correctamente.")
    return df_resultados

# Ejecutar después de resolver el modelo:
df = exportar_resultados_vehiculos_case3(Model, distancias, D_demanda, V_capacidad, V_autonomia, E_costo, L, D, E, V, costo_peaje)
from IPython.display import display

display(df)

Archivo verificacion_caso3.csv exportado correctamente.


,VehicleId,LoadCap,FuelCap,RouteSeq,Municipalities,Demand,InitLoad,InitFuel,RefuelStops,RefuelAmounts,TollsVisited,TollCosts,VehicleWeights,Distance,Time,FuelCost,TollCost,TotalCost
0,CAM001,80.0,1720,PTO - MUN07 - MUN06 - EST19 - EST23 - MUN03 - ...,5,17-15-18-10-17,78.2,1720,0,0,0,0,17-15-18-10-17,1300.3,26.01,0,0,7411940
1,CAM002,60.0,1510,PTO - MUN10 - PTO,1,11,11.0,1510,0,0,0,0,11,248.7,4.97,0,0,1417457
2,CAM003,50.0,1300,PTO - MUN11 - EST16 - MUN09 - EST18 - MUN13 - ...,4,9-10-7-18,44.0,1300,0,0,0,0,9-10-7-18,1828.1,36.56,0,0,10420120
3,CAM004,40.0,1500,PTO - MUN02 - PTO,1,16,16.0,1500,0,0,0,0,16,19.4,0.39,0,0,110781
4,CAM005,30.0,870,PTO - MUN12 - EST22 - MUN04 - PTO,2,5-16,21.0,870,0,0,0,0,5-16,687.9,13.76,0,0,3921262
5,CAM006,10.0,1200,PTO - MUN14 - PTO,1,5,5.0,1200,0,0,0,0,5,177.1,3.54,0,0,1009550


In [8]:
def visualizar_rutas_en_mapa(df_resultados, locations_df):
    locations_dict = {
        int(row['LocationID']): (row['Latitude'], row['Longitude']) for _, row in locations_df.iterrows()
    }

    for idx, row in df_resultados.iterrows():
        mapa = folium.Map(location=[4.5709, -74.2973], zoom_start=6)
        secuencia = row['RouteSeq'].split(' - ')

        coordenadas = []
        for s in secuencia:
            if s == 'PTO':
                numero = 1
            elif s.startswith('MUN') or s.startswith('EST') or s.startswith('PEA'):
                numero = int(s[-2:])
            else:
                continue
            coord = locations_dict.get(numero)
            if coord:
                coordenadas.append(coord)

        folium.PolyLine(coordenadas, color="blue", weight=4.5, opacity=1).add_to(mapa)

        for i, coord in enumerate(coordenadas):
            folium.Marker(coord, popup=secuencia[i]).add_to(mapa)

        display(mapa)

visualizar_rutas_en_mapa(df, locations_csv)

In [12]:
def resumen_por_vehiculo(df_resultados, V_capacidad):
    resumen = []

    for idx, row in df_resultados.iterrows():
        vehiculo = row['Vehiculo'] if 'Vehiculo' in row else row.name
        ruta = row['RouteSeq'].split(' - ')
        estaciones = [s for s in ruta if s.startswith('EST')]
        municipios = [s for s in ruta if s.startswith('MUN') or s == 'PTO']

        resumen.append({
            'Vehículo': vehiculo,
            'Ruta': row['RouteSeq'],
            'Distancia Total (km)': round(row.get('DistanciaTotal', 0), 2),
            'Tiempo Estimado (h)': round(row.get('DistanciaTotal', 0) / 50, 2),
            'Peajes ($)': round(row.get('Peajes', 0), 2),
            'Mantenimiento ($)': round(row.get('Mantenimiento', 0), 2),
            'Combustible ($)': round(row.get('Combustible', 0), 2),
            'Costo Total ($)': round(row.get('CostoTotal', 0), 2),
            'Estaciones de Recarga': len(estaciones),
            'Municipios Visitados': len(municipios),
            'Peso Transportado (kg)': round(row.get('PesoTotal', 0), 2),
            'Capacidad Vehículo (kg)': V_capacidad.get(int(vehiculo), 0)
        })

    df_resumen = pd.DataFrame(resumen)
    display(df_resumen)
    return df_resumen


df_resumen = resumen_por_vehiculo(df, V_capacidad)

,Vehículo,Ruta,Distancia Total (km),Tiempo Estimado (h),Peajes ($),Mantenimiento ($),Combustible ($),Costo Total ($),Estaciones de Recarga,Municipios Visitados,Peso Transportado (kg),Capacidad Vehículo (kg)
0,0,PTO - MUN07 - MUN06 - EST19 - EST23 - MUN03 - ...,0,0.0,0,0,0,0,4,7,0,0.0
1,1,PTO - MUN10 - PTO,0,0.0,0,0,0,0,0,3,0,80.0
2,2,PTO - MUN11 - EST16 - MUN09 - EST18 - MUN13 - ...,0,0.0,0,0,0,0,5,6,0,60.0
3,3,PTO - MUN02 - PTO,0,0.0,0,0,0,0,0,3,0,50.0
4,4,PTO - MUN12 - EST22 - MUN04 - PTO,0,0.0,0,0,0,0,1,4,0,40.0
5,5,PTO - MUN14 - PTO,0,0.0,0,0,0,0,0,3,0,30.0


## ANALISIS CASO 3